In [86]:
!pip install plotly -q
!pip install scrapy -q

# Plan your TRIP !

projet kayak

Le projet est de développer un nouvel outil qui permettrait aux utilisateurs désireux de planifier un voyage d'avoir plus d'informations quant à leur destinations. Et oui, qui n'a pas envie à la dernière minute d'une petite escapade improvisée ?... à condition de faire le bon choix et de ne pas passer des heures de recherche !
Nous allons donc fournir à nos utilisateurs les données météos des prochains jours afin de bien choisir sa destination, avec le meilleur hôtel qui soit. De quoi mettre toutes les chances de son côté afin de se façonner le meilleur des week end !
Pour cela 35 destinations dans toute la France pourront être proposées !

ATTENTION : ce projet a été réalisé fin aout 2021 - septembre 2021 : il semblerait que booking ait modifié l'emplacement des données depuis, elles ne sont plus accessibles de la même manière, le code en fin de script ne correspond plus.

## Import librairie et premières manipulations

In [65]:
import requests
import json 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os 
import logging
from datetime import datetime
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "iframe"

Ici, nous allons déjà essayer de voir quelles informations météo, nous pourrions utiliser, lesquelles sont pouvons-nous trouver :

In [3]:
villes = ["Mont Saint Michel","St Malo","Bayeux", "Le Havre", "Rouen", "Paris", "Amiens","Lille","Strasbourg""Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille","Aix en Provence","Avignon","Uzes","Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne","Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [4]:
# Nous allons tout d'abord faire des premières recherches sur la première ville de la liste, afin d'étudier les options faisables pour la réalisation de notre application.
localisation = requests.get('https://nominatim.openstreetmap.org/search?city= Mont Saint Michel&format=json')

In [5]:
# Recherche de la localisation du Mont St Michel
localisation.json()[0]

{'place_id': 151486647,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 211285890,
 'boundingbox': ['48.6349172', '48.637031', '-1.5133292', '-1.5094796'],
 'lat': '48.6359541',
 'lon': '-1.511459954959514',
 'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
 'class': 'place',
 'type': 'islet',
 'importance': 0.755436556781574}

In [6]:
lat=localisation.json()[0]['lat']
lon=localisation.json()[0]['lon']
name=villes[0]

In [7]:
name

'Mont Saint Michel'

In [8]:
# Grâce à openweathermap, nous allons donc rechercher via l'API de ce dernier, les informations que nous pouvons collecter sur notre ville de recherche
weather=requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=hourly,minutely&units=metric&appid=4aa16d79efdfe21e66e31b4aec2300bf")

In [9]:
weather

<Response [200]>

In [10]:
# Ci-dessous, toutes les options que nous pouvons trouver sur notre ville concernant la météo, de quoi donner des indications précises à nos utilisateurs.
weather.json()

{'lat': 48.636,
 'lon': -1.5115,
 'timezone': 'Europe/Paris',
 'timezone_offset': 3600,
 'current': {'dt': 1641993871,
  'sunrise': 1641974115,
  'sunset': 1642005175,
  'temp': 9.16,
  'feels_like': 9.16,
  'pressure': 1039,
  'humidity': 90,
  'dew_point': 7.61,
  'uvi': 0.48,
  'clouds': 100,
  'visibility': 10000,
  'wind_speed': 0.99,
  'wind_deg': 80,
  'wind_gust': 1.79,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}]},
 'daily': [{'dt': 1641988800,
   'sunrise': 1641974115,
   'sunset': 1642005175,
   'moonrise': 1641990960,
   'moonset': 1641956340,
   'moon_phase': 0.33,
   'temp': {'day': 9.12,
    'min': 6.27,
    'max': 9.42,
    'night': 6.83,
    'eve': 9.01,
    'morn': 7.21},
   'feels_like': {'day': 9.12, 'night': 4.72, 'eve': 7.42, 'morn': 7.21},
   'pressure': 1039,
   'humidity': 90,
   'dew_point': 7.57,
   'wind_speed': 3.39,
   'wind_deg': 68,
   'wind_gust': 6.63,
   'weather': [{'id': 500,
     'main': 

In [85]:
# Nous allons ici extraire les informations disponibles pour la prochaine semaine...
day=0
climate=[]
degre=[]
for i in range (0,8):
    climate.append(weather.json()['daily'][day]['weather'][0]['main'])
    degre.append(weather.json()['daily'][day]['feels_like']['day'])
    day+=1
data = climate,degre

#

In [12]:
dataset=pd.DataFrame(data)

dataset.rename(index={0:name +'/ weather',1:name +'/ temperature'})

,0,1,2,3,4,5,6,7
Mont Saint Michel/ weather,Rain,Clear,Clear,Clear,Clouds,Clouds,Clear,Clear
Mont Saint Michel/ temperature,9.12,6.78,5.72,4.02,6,7.21,5.81,4.05


## Scraping avec booking.com

Maintenant que nous avons pu avoir un premier aperçu des informations que nous pouvons obtenir, nous allons généraliser cette recherche à nos 35 villes, en récupérant également leur longitude et latitude, puis nous allons nous servir du site booking.com afin de récuperer une liste d'hôtel pour chaque ville, via le scraping...

In [14]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [87]:
#liste des 35 meilleures villes  selectionnée selon One Week In.com
liste_ville = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens",
               "Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon",
               "Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis","Marseille",
                "Aix en Provence","Avignon","Uzes",
                "Nimes","Aigues Mortes","Saintes Maries de la mer","Collioure","Carcassonne",
                "Ariege","Toulouse","Montauban","Biarritz","Bayonne","La Rochelle"]

In [88]:
# liste de la longitude et la latitude des 35 villes selectionnées 
#grâce à la librairie requests
liste_gps = []

for element in liste_ville:

    #API
    r_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(element)) 
    dic_gps = r_gps.json()
    latitude = float(dic_gps[0]["lat"])
    longitude = float(dic_gps[0]["lon"])
    liste_gps.append([latitude,longitude])

In [89]:
#on crée un ID pour ajouter au datafram et on fusionne les datafram plus tard
ID = [i+1 for i in range(len(liste_ville))]

In [90]:
# on crée un datafram contenant les 35 villes et la géolocalisation ainsi que leurs ID
ID = [i for i in range(len(liste_ville))]
dataset = pd.DataFrame(liste_gps)
dataset['ville']= liste_ville
dataset ['ID']=ID
dataset.columns = ['longitude','latitude', 'villes','ID']
#display(dataset)

In [91]:
# on recupère la météo des 35 villes selectionnées sur 7 jours
# on recupère la météo des 35 villes selectionnées sur 7 jours
liste_meteo=[]

for x in range(len(liste_ville)):
    ville = liste_ville[x]
    Id= x
    latitude1 = liste_gps[x][0]
    longitude2 = liste_gps[x][1]

    r_meteo = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&exclude=hourly,minutely&units=metric&appid=4aa16d79efdfe21e66e31b4aec2300bf")
    dic_meteo = r_meteo.json()
    for méteo in range(1,8):
        lat=float(dic_meteo['lat'])
        lon=float(dic_meteo['lon'])
        date_scrap = dic_meteo['daily'][méteo]['dt']
        date = datetime.fromtimestamp (date_scrap)
        dates = date.strftime("%d/%m/%y")
        temps = dic_meteo["daily"][méteo]["weather"][0]["description"]
        temperature_max = float (dic_meteo["daily"][méteo]['temp']['max'])
        temperature_min = float (dic_meteo["daily"][méteo]['temp']['min'])
        temperature_jour = float(dic_meteo["daily"][méteo]['temp']['day'])
        liste_meteo.append([lat,lon,ville,Id,dates,temps,temperature_max,temperature_min,temperature_jour])

In [92]:
dataset2= pd.DataFrame(liste_meteo)
dataset2.columns = ['lat','lon','Nom_Ville','Id','dates','temps','temperature_max','temperature_min','temperature_jour']

In [93]:
# je fais une analyse des villes les plus chaudes en calculant la moyenne 
#des températures des plus chaudes sur 7 jours qui est mon critère pour définir les meilleures vacances
avg_tmax = dataset2.groupby(['Nom_Ville'])['temperature_max'].mean()
avg_tmax = avg_tmax.sort_values(ascending = False)
avg_tmax = pd.DataFrame(avg_tmax)
fusion = pd.merge(dataset, avg_tmax, how="right", left_on="villes", right_on="Nom_Ville")
fusion

,longitude,latitude,villes,ID,temperature_max
0,44.012128,4.419672,Uzes,23,8.495714
1,47.238022,6.024362,Besancon,12,8.495714
2,42.525050,3.083155,Collioure,27,8.495714
3,48.249490,7.344296,Chateau du Haut Koenigsbourg,9,8.495714
4,43.214036,5.539632,Cassis,19,8.495714
5,43.213036,2.349107,Carcassonne,28,8.495714
6,43.157217,6.329254,Bormes les Mimosas,18,8.495714
7,43.471144,-1.552727,Biarritz,32,8.495714
8,43.493338,-1.475099,Bayonne,33,8.495714
9,47.321581,5.041470,Dijon,13,8.495714


Scrapping

In [73]:
#on récupère 20 hotels ainsi que leurs informations en fonction de nos 35 villes selectionnées précédemment 
#grâce a beautifulsoup
liste_booking=[]

#soup = BeautifulSoup("<html>data</html>", "html.parser")

for hotels in range(len(liste_ville)):
    villes = liste_ville[hotels]
    Id= hotels
       
    r_booking = requests.get("https://www.booking.com/searchresults.fr.html?ss={}".format(liste_ville[hotels]), headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    soup = BeautifulSoup(r_booking.content)
    link=soup.select("div.b_hotelSummary")

    adresse_hotel = [element.get_text().replace('\n','') for element in soup.find_all("p","b_hotelAddress")]
    hotel_description = [element.get_text().replace('\n','') for element in soup.find_all("p","hotel_description")]
    hotels_url = [x.get('href') for x in soup.select("h3 a")] 
    nom_hotel = [x.get('title') for x in soup.select('div.hotel_information a img[title]')]
    hotels_score =[x.get('alt') if x.get('alt') != "" else '0' for x in soup.select('div.hotel_review_score b img[alt]')]
    for explode in range(len(link)):
        data = {
        "ID":Id,
        "villes":villes,
        "name_hotels":nom_hotel[explode], 
        "adresse_hotel":adresse_hotel[explode],
        "hotels_url":hotels_url[explode],
        "hotel_description":hotel_description[explode],
        "hotels_score":hotels_score
    }
    
        liste_booking.append(data)

data=pd.DataFrame(liste_booking)
    
        #data["hotels_score"] = data["hotels_score"].apply(lambda x: if = x.append(np.nan))
    
    #for row in data.iterrows:
        #print(row["ID"], row["villes"],row["name_hotels"], row["adresse_hotel"],row["hotels_url"], row["hotel_description"],row["hotels_score"])

In [74]:
data

,ID,villes,name_hotels,adresse_hotel,hotels_url,hotel_description,hotels_score
0,0,Mont Saint Michel,Hôtel Vert,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/vert.fr.html?label=gen173nr-1FCAQogg...,"Situé à 2 km du Mont-Saint-Michel, sur la côte...",[]
1,0,Mont Saint Michel,Le Relais Saint Michel,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/le-relais-saint-michel.fr.html?label...,Le Relais Saint Michel vous accueille face à l...,[]
2,0,Mont Saint Michel,Les Terrasses Poulard,"Grande Rue, Le Mont-Saint-Michel",/hotel/fr/les-terrasses-poulard.fr.html?label=...,Occupant 2 bâtiments différents au cœur du Mon...,[]
3,0,Mont Saint Michel,Hotel De La Digue,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/de-la-digue.fr.html?label=gen173nr-1...,L'hôtel De La Digue est un établissement tradi...,[]
4,0,Mont Saint Michel,Le Mouton Blanc,"Grande Rue , Le Mont-Saint-Michel",/hotel/fr/le-mouton-blanc-le-mont-saint-michel...,"Situé au pied de l'abbaye, le Mouton Blanc Hot...",[]
5,0,Mont Saint Michel,Mercure Mont Saint Michel,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/mont-saint-michel.fr.html?label=gen1...,Installé dans des espaces verts à seulement 2 ...,[]
6,0,Mont Saint Michel,La Mère Poulard,"Grande Rue, Le Mont-Saint-Michel",/hotel/fr/la-mere-poulard.fr.html?label=gen173...,"Occupant un bâtiment historique, l'hôtel La Mè...",[]
7,0,Mont Saint Michel,Auberge Saint Pierre,"Grande Rue, Le Mont-Saint-Michel",/hotel/fr/auberge-saint-pierre.fr.html?label=g...,L'Auberge Saint-Pierre occupe une maison à col...,[]
8,0,Mont Saint Michel,Hotel Gabriel,"Route du Mont Saint Michel, Le Mont-Saint-Michel",/hotel/fr/hotel-gabriel.fr.html?label=gen173nr...,"L’Hotel Gabriel se trouve à 1,6 km du Mont-Sai...",[]
9,0,Mont Saint Michel,Le Relais Du Roy,"La Caserne, Le Mont-Saint-Michel",/hotel/fr/le-relais-du-roy.fr.html?label=gen17...,Le Relais Du Roy est un hôtel 3 étoiles situé ...,[]


In [75]:
liste_url = data['hotels_url'].tolist()

In [80]:
# Nous récupérons la géolocalisation des hotels grâce à l'url de chaque hotel:
geo_hotel = []

for hotels in range(len(liste_url)):

       
    r_geo = requests.get("https://www.booking.com{}".format(liste_url[hotels]), headers = {'User-Agent': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
    soup = BeautifulSoup(r_geo.content)
    link = soup.select("a.show_map")
    
    name_hotel = [x.get('title') for x in soup.select('#b_google_map_thumbnail img')]
    geo = [x.get('data-coords') for x in soup.select("#show_map")][0]
    long = float(geo.split(",")[0])
    lati = float(geo.split(",")[1])

    geo_hotel.append([lati,long, name_hotel])

TooManyRedirects: Exceeded 30 redirects.

In [ ]:
dataset3= pd.DataFrame(geo_hotel)
dataset3.columns=['latitude','longitude','name_hotel']
result = pd.concat([data, dataset3], axis=1).reindex(data.index)
#result1 = pd.merge(villes, name_hotel, how="right", left_on="name_hotel", right_on="name_hotels")

In [ ]:
#Et nous pouvons donc maintenant créer une carte qui affiche la météo des 35 villes :
list_column = ['villes']


fig = px.scatter_mapbox(fusion,
                        lat="longitude",
                        lon="latitude",
                        color="temperature_max",
                        size="temperature_max",
                        zoom=10,
                        #color_discrete_sequence=["red"],
                        #text='Villes',
                        title='Temperature moyenne',
                        mapbox_style="carto-positron",
                        hover_data=list_column,)
fig.show()

In [77]:
#je crée une carte qui affiche les 20 hotels des 35 villes
liste_column = ['villes', 'name_hotels']

fig = px.scatter_mapbox(result,
                        lat='latitude',
                        lon="longitude",
                        color="ID",
                        size=result["latitude"],
                        zoom=10,
                        #color_discrete_sequence=["red"],
                        #text='Villes',
                        title='liste hotel france',
                        mapbox_style="carto-positron",
                        hover_data=liste_column,)
fig.show()

NameError: name 'result' is not defined

In [ ]:
dataset.to_csv('geolocalisation.csv')
dataset2.to_csv('meteo.csv')
fusion.to_csv('avg_meteo.csv')
result.to_csv('hotel_booking.csv',encoding='utf-8',index=False, sep="\t")

## Data lake S bucket

In [48]:
!pip install boto3 -q

In [ ]:
import boto3

# Nous allons tout d'abord configurer notre notre session d'utilisateur :
MY_acces_KEY = "***********"
MY_secret_KEY = "**********"

session = boto3.Session(aws_access_key_id= MY_acces_KEY, 
                        aws_secret_access_key= MY_secret_KEY)

In [ ]:
# Nous sommes donc prêt à pusher nos information
s3 = session.resource("s3")
bucket = s3.create_bucket(Bucket="plan_your_trip")

In [ ]:
import pandas as pd

data = pd.read_csv('meteo.csv')
csv = data.to_csv()
put_object = bucket.put_object(Key="meteo.csv", Body=csv)

In [ ]:
data = pd.read_csv('hotel_booking.csv',sep="\s{3,}", engine='python')
csv = data.to_csv()
put_object = bucket.put_object(Key="hotel_bookink.csv", Body=csv)

In [ ]:
data = pd.read_csv('avg_meteo.csv')
csv = data.to_csv()
put_object = bucket.put_object(Key="avg_meteo.csv", Body=csv)

## Data Warehouse

Enfin, grâce à AWS RDS, nous pouvons créer également créer une base de données qui permettra d'effectuer les requêtes adéquates sur des données propres :

In [ ]:
!pip install psycopg2-binary -q
from sqlalchemy import create_engine

engine = create_engine("postgresql+psycopg2://********:*********@************/postgres", echo=True)

In [ ]:
meteo = pd.read_csv('meteo.csv')

In [ ]:
meteo.to_sql('meteo_7days_35villes', engine)